### defining parameters and importing libraries

In [9]:
import pandas as pd
import numpy as np
from math import  inf
from docplex.mp.model import Model
#parameters
yields={1:[3,2.5,2,3,2.5,2,3,2.5,2] , 2:[3.6,3,2.4,3.6,3,2.4,3.6,3,2.4] , 3:[24,20,16,24,20,16,24,20,16]}
planting_cost=[150,230,260]
buying_price=[238,210]
selling_price=[170, 150, 36, 10]


In [14]:
# i is our counter which we can use to set how many loop we want our algorithm to Run
i=1
#define E21 and e21
#these variables are used to produce cuts in each iteration
E21=np.zeros([5,3])
e21=np.zeros(5)
E11=np.zeros([5,3])
e11=np.zeros(5)
while i<=5:
    print('##############################################################################################','\n',
    '###########################################{}تکرار#######################################'.format(i),'\n',
    '#############################################################################################')
    NLDS1=Model(name='NLDS1')
    x1=NLDS1.continuous_var_matrix(range(1,4),range(1,3),name='x',key_format='%s')
    teta1=NLDS1.continuous_var_list(range(1,2),lb=-inf,name='teta1',key_format='%s')
    NLDS1.add_constraint_(x1[1,1]+x1[2,1]+x1[3,1]<=500)
    if i==1:
        NLDS1.add_constraint_(teta1[0]==0,'teta==0 constraint')
    if i>1:
        for k in range(i-1):
            NLDS1.add_constraint_(x1[1,1] * E11[k][0] + x1[2,1] * E11[k][1] + x1[3,1] * E11[k][2] + teta1[0] >= e11[k])

    NLDS1.minimize(sum(planting_cost[i-1]*x1[i,j] for i in range(1,4) for j in range(1,2))+teta1[0])
    sol_NLDS1=NLDS1.solve()
    sol_NLDS1.display()
    pi_NLDS2={}
    for s in range(1,4):
        NLDS2=Model(name='NLDS2{}'.format(s))
        #variables
        x2=NLDS2.continuous_var_matrix(range(1,4),range(1,3),name='x',key_format='%s')
        teta2=NLDS2.continuous_var_list(range(1,4),lb=-inf,name='teta2',key_format='%s')
        y2=NLDS2.continuous_var_cube(range(1,3),range(1,4),range(2,4),name='y',key_format='%s')
        w2=NLDS2.continuous_var_cube(range(1,5),range(1,4),range(2,4),name='w',key_format='%s')
        #constraints
        NLDS2.add_constraint_(-x2[1,2]-x2[2,2]-x2[3,2]>=-500,'1')
        NLDS2.add_constraint_(sol_NLDS1[x1[1,1]] * yields[1][s-1] + y2[1,s,2] - w2[1,s,2] >= 200,'2')
        NLDS2.add_constraint_(sol_NLDS1[x1[2,1]] * yields[2][s-1] + y2[2,s,2] - w2[2,s,2] >= 240,'3')
        NLDS2.add_constraint_(sol_NLDS1[x1[3,1]] * yields[3][s-1] - w2[3,s,2] - w2[4,s,2] >= 0,'4')
        NLDS2.add_constraint_(-w2[3,s,2]>=-6000,'5')
        NLDS2.add_constraint_(sol_NLDS1[x1[1,1]] + sol_NLDS1[x1[2,1]] - x2[3,2]>=0,'6')
        if i==1:
                NLDS2.add_constraint_(teta2[s-1]==0,'teta==0 constraint')
        if i>1:
            for k in range(i-1):
                NLDS2.add_constraint_(E21[k][0] * x2[1,2] + E21[k][1] * x2[2,2] + E21[k][2] * x2[3,2] + teta2[s-1] >= e21[k])
    
        #objective function
        NLDS2.minimize(150*x2[1,2]+230*x2[2,2]+260*x2[3,2]+sum(buying_price[i-1]*y2[i,s,2] for i in range(1,3))+
                    sum(selling_price[j-1]*w2[j,s,2] for j in range(1,5))+teta2[s-1])   
        sol_NLDS2=NLDS2.solve()
        sol_NLDS2.display()
        simplex=[]
        simplex.append(NLDS2.dual_values(NLDS2.find_matching_linear_constraints('1'))[0])
        simplex.append(NLDS2.dual_values(NLDS2.find_matching_linear_constraints('2'))[0])
        simplex.append(NLDS2.dual_values(NLDS2.find_matching_linear_constraints('3'))[0])
        simplex.append(NLDS2.dual_values(NLDS2.find_matching_linear_constraints('4'))[0])
        simplex.append(NLDS2.dual_values(NLDS2.find_matching_linear_constraints('5'))[0])
        simplex.append(NLDS2.dual_values(NLDS2.find_matching_linear_constraints('6'))[0])
        pi_NLDS2['NLDS2{}'.format(s)]=simplex
    print('simplex multipliers for NLDS2',pi_NLDS2)
    pi_NLDS3={}
    for s in range(1,10):
        NLDS3=Model(name='NLDS3{}'.format(s))
        #variables
        teta3=NLDS3.continuous_var_list(range(1,10),lb=-inf,name='teta2',key_format='%s')
        y3=NLDS3.continuous_var_cube(range(1,3),range(1,10),range(2,4),name='y',key_format='%s')
        w3=NLDS3.continuous_var_cube(range(1,5),range(1,10),range(2,4),name='w',key_format='%s')
        #constraints
        NLDS3.add_constraint_(sol_NLDS2[x2[1,2]] * yields[1][s-1] + y3[1,s,3] - w3[1,s,2] >= 200,'1')
        NLDS3.add_constraint_(sol_NLDS2[x2[2,2]] * yields[2][s-1] + y3[2,s,3] - w3[2,s,3] >= 240,'2')
        NLDS3.add_constraint_(sol_NLDS2[x2[3,2]] * yields[3][s-1] - w3[3,s,3] - w3[4,s,3] >= 0,'3')
        NLDS3.add_constraint_(-w3[3,s,3]>=-6000,'4')
        NLDS3.add_constraint_(teta3[s-1]==0)
        #objective function
        NLDS3.minimize(sum(buying_price[i-1]*y3[i,s,3] for i in range(1,3))+
                    sum(selling_price[j-1]*w3[j,s,3] for j in range(1,5))+teta3[s-1])
        sol_NLDS3=NLDS3.solve()
        sol_NLDS3.display()
        simplex=[]
        simplex.append(NLDS3.dual_values(NLDS3.find_matching_linear_constraints('1'))[0])
        simplex.append(NLDS3.dual_values(NLDS3.find_matching_linear_constraints('2'))[0])
        simplex.append(NLDS3.dual_values(NLDS3.find_matching_linear_constraints('3'))[0])
        simplex.append(NLDS3.dual_values(NLDS3.find_matching_linear_constraints('4'))[0])
        pi_NLDS3['NLDS3{}'.format(s)]=simplex
    print('simplex multipliers for NLDS3',pi_NLDS3)
    ##########################################################################################################################
    ############################################### BACKWARD DIRECTION ########################################################
    ##########################################################################################################################
    #t-1=2
    T21=np.array([[3,0,0],[0,3.6,0],[0,0,24],[0,0,0]])
    T22=np.array([[2.5,0,0],[0,3,0],[0,0,20],[0,0,0]])
    T23=np.array([[2,0,0],[0,2.4,0],[0,0,16],[0,0,0]])
    pi31=np.array(pi_NLDS3['NLDS31']).reshape(1,4)
    pi32=np.array(pi_NLDS3['NLDS32']).reshape(1,4)
    pi33=np.array(pi_NLDS3['NLDS33']).reshape(1,4)
    E21[i-1]=(1/3)*np.matmul(pi31,T21)+(1/3)*np.matmul(pi32,T22)+(1/3)*np.matmul(pi33,T23)
    print('E21:',E21[i-1])
    #E22 and E23 will be the same as E21
    h31=np.array([200,240,0,-6000]).reshape(4,1)
    h32=h31
    h33=h31
    e21[i-1]=(1/3)*np.matmul(pi31,h31)+(1/3)*np.matmul(pi32,h32)+(1/3)*np.matmul(pi33,h33)
    print('e21:',e21[i-1])
    x21=[sol_NLDS2[x2[1,2]] ,sol_NLDS2[x2[2,2]] ,
    sol_NLDS2[x2[3,2]]]).reshape(3,1)
    if e21[i-1]-np.matmul(E21[i-1],x21)>=sol_NLDS2[teta2[0]]:
        print('condition does not hold - we add a cut to NLDS21 problem')
    else:
        print('no cut needed go to t-1')
    #########################################
    for s in range(1,4):
        NLDS2=Model(name='NLDS2{}'.format(s))
        #variables
        x2=NLDS2.continuous_var_matrix(range(1,4),range(1,3),name='x',key_format='%s')
        teta2=NLDS2.continuous_var_list(range(1,4),lb=-inf,name='teta2',key_format='%s')
        y2=NLDS2.continuous_var_cube(range(1,3),range(1,4),range(2,4),name='y',key_format='%s')
        w2=NLDS2.continuous_var_cube(range(1,5),range(1,4),range(2,4),name='w',key_format='%s')
        #constraints
        NLDS2.add_constraint_(-x2[1,2]-x2[2,2]-x2[3,2]>=-500,'1')
        NLDS2.add_constraint_(sol_NLDS1[x1[1,1]] * yields[1][s-1] + y2[1,s,2] - w2[1,s,2] >= 200,'2')
        NLDS2.add_constraint_(sol_NLDS1[x1[2,1]] * yields[2][s-1] + y2[2,s,2] - w2[2,s,2] >= 240,'3')
        NLDS2.add_constraint_(sol_NLDS1[x1[3,1]] * yields[3][s-1] - w2[3,s,2] - w2[4,s,2] >= 0,'4')
        NLDS2.add_constraint_(-w2[3,s,2]>=-6000,'5')
        NLDS2.add_constraint_(sol_NLDS1[x1[1,1]] + sol_NLDS1[x1[2,1]] - x2[3,2]>=0,'6')
        for x in range(i):
                NLDS2.add_constraint_(E21[x][0] * x2[1,2] + E21[x][1] * x2[2,2] + E21[x][2] * x2[3,2] + teta2[s-1] >= e21[x])
        #objective function
        NLDS2.minimize(150*x2[1,2]+230*x2[2,2]+260*x2[3,2]+sum(buying_price[i-1]*y2[i,s,2] for i in range(1,3))+
                    sum(selling_price[j-1]*w2[j,s,2] for j in range(1,5))+teta2[s-1])
        NLDS2.solve().display()
        simplex=[]
        simplex.append(NLDS2.dual_values(NLDS2.find_matching_linear_constraints('1'))[0])
        simplex.append(NLDS2.dual_values(NLDS2.find_matching_linear_constraints('2'))[0])
        simplex.append(NLDS2.dual_values(NLDS2.find_matching_linear_constraints('3'))[0])
        simplex.append(NLDS2.dual_values(NLDS2.find_matching_linear_constraints('4'))[0])
        simplex.append(NLDS2.dual_values(NLDS2.find_matching_linear_constraints('5'))[0])
        simplex.append(NLDS2.dual_values(NLDS2.find_matching_linear_constraints('6'))[0])
        pi_NLDS2['NLDS2{}'.format(s)]=simplex
    #t-1=1
    T11=np.array([[3,0,0],[0,3.6,0],[0,0,24],[0,0,0],[0,0,0],[0,0,0]])
    T12=np.array([[2.5,0,0],[0,3,0],[0,0,20],[0,0,0],[0,0,0],[0,0,0]])
    T13=np.array([[2,0,0],[0,2.4,0],[0,0,16],[0,0,0],[0,0,0],[0,0,0]])
    pi21=np.array(pi_NLDS2['NLDS21']).reshape(1,6)
    pi22=np.array(pi_NLDS2['NLDS22']).reshape(1,6)
    pi23=np.array(pi_NLDS2['NLDS23']).reshape(1,6)
    E11[i-1]=(1/3)*np.matmul(pi21,T11)+(1/3)*np.matmul(pi22,T12)+(1/3)*np.matmul(pi23,T13)
    print('E11:',E11[i-1])
    #E22 and E23 will be the same as E21
    h21=np.array([-500,200,240,0,-6000,0]).reshape(6,1)
    h22=h21
    h23=h21
    e11[i-1]=(1/3)*np.matmul(pi21,h21)+(1/3)*np.matmul(pi22,h22)+(1/3)*np.matmul(pi23,h23)
    print('e11:',e11[i-1])
    x11=[sol_NLDS1[x1[1,1]] ,sol_NLDS1[x1[2,1]] ,
    sol_NLDS1[x1[3,1]]]).reshape(3,1)
    if e21[i-1]-np.matmul(E11[i-1],x11)>=sol_NLDS1.get_value_list(teta1)[0]:
        print('condition does not hold - we add a cut to NLDS1 problem')
    else:
        print('no cut needed - optimal solution achieved!')
        break
    NLDS1.add_constraint_(x1[1,1] * E11[i-1][0] + x1[2,1] * E11[i-1][1] + x1[3,1] * E11[i-1][2] + teta1[0] >= e11[i-1])
    NLDS1.remove_constraint('teta==0 constraint')
    sol_NLDS1=NLDS1.solve()
    sol_NLDS1.display()



    i+=1





SyntaxError: unmatched ')' (Temp/ipykernel_19240/667394135.py, line 93)

In [12]:
NLDS2.dual_values(NLDS2.find_matching_linear_constraints('a'))[0]

0